## 第一個網路爬蟲

In [2]:
import requests
res = requests.get('https://www.mof.gov.tw/Pages/Detail.aspx?nodeid=281&pid=72221', verify=False)

C:\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [8]:
#print(res.text)
print(type(res))

<class 'requests.models.Response'>


## 使用BeautifulSoup4

In [6]:
html_sample = '''
<html> 
 <body> 
 <h1 id="title">Hello World</h1> 
 <a href="http://www.largitdata.com" class="link">This is link1</a> 
 <a href="# link2" class="link">This is link2</a> 
 </body> 
</html>
'''

In [19]:
name = 'David'
names = ['David', 'Marry', 'John']
print(names[0])
print(names[2])

David
John


In [9]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_sample, 'html.parser')
#print(soup.text)
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [17]:
li = soup.select('h1')
print(li)
print(type(li))
print(li[0])
print(type(li[0]))
print(li[0].text)

[<h1 id="title">Hello World</h1>]
<class 'list'>
<h1 id="title">Hello World</h1>
<class 'bs4.element.Tag'>
Hello World


In [23]:
a = soup.select('a')
print(a)
print(type(a))
print(a[1])
print(type(a[1]))
print(a[1].text)

[<a class="link" href="http://www.largitdata.com">This is link1</a>, <a class="link" href="# link2">This is link2</a>]
<class 'list'>
<a class="link" href="# link2">This is link2</a>
<class 'bs4.element.Tag'>
This is link2


In [27]:
for a in soup.select('a'):
    #print(a)
    #print(a.text)
    print(a['href'])

http://www.largitdata.com
# link2


## 根據CSS 屬性選擇

In [29]:
# id => #
soup.select('#title')

[<h1 id="title">Hello World</h1>]

In [32]:
# class => .
soup.select('.link')

[<a class="link" href="http://www.largitdata.com">This is link1</a>,
 <a class="link" href="# link2">This is link2</a>]

## 取得財政部Excel 連結

In [34]:
import requests
res = requests.get('https://www.mof.gov.tw/Pages/Detail.aspx?nodeid=281&pid=72221', verify=False)
#res.text

C:\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py:821: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


In [40]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'html.parser')
for link in soup.select('.1 a'):
    print(link['href'])

http://www.mof.gov.tw/File/Attach/72221/File_8550.xls
http://www.mof.gov.tw/File/Attach/72221/File_8549.ods
http://www.mof.gov.tw/File/Attach/72221/File_8552.xls
http://www.mof.gov.tw/File/Attach/72221/File_8551.ods
http://www.mof.gov.tw/File/Attach/72221/File_8554.xls
http://www.mof.gov.tw/File/Attach/72221/File_8553.ods
http://www.mof.gov.tw/File/Attach/72221/File_8556.xls
http://www.mof.gov.tw/File/Attach/72221/File_8555.ods
http://www.mof.gov.tw/File/Attach/72221/File_8558.xls
http://www.mof.gov.tw/File/Attach/72221/File_8557.ods
http://www.mof.gov.tw/File/Attach/72221/File_8560.xls
http://www.mof.gov.tw/File/Attach/72221/File_8559.ods
http://www.mof.gov.tw/File/Attach/72221/File_8562.xls
http://www.mof.gov.tw/File/Attach/72221/File_8561.ods
http://www.mof.gov.tw/File/Attach/72221/File_8564.xls
http://www.mof.gov.tw/File/Attach/72221/File_8563.ods
http://www.mof.gov.tw/File/Attach/72221/File_8566.xls
http://www.mof.gov.tw/File/Attach/72221/File_8565.ods
http://www.mof.gov.tw/File/A

In [47]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'html.parser')
for link in soup.select('.1 a'):
    fileurl = link['href']
    res2 = requests.get(fileurl)
    filename = fileurl.split('/')[-1]
    with open(filename, 'wb') as f:
        f.write(res2.content)

## Download Excel

In [41]:
import requests
res2 = requests.get('http://www.mof.gov.tw/File/Attach/72221/File_8550.xls')
with open('File_8550.xls', 'wb') as f:
    f.write(res2.content)

## Sample Parsing

In [52]:
import pandas
df = pandas.read_excel('File_8550.xls', skiprows=range(0,5), skip_footer=4)
df.columns = ['yearmonth','importation','importation_annual','exportation','exportation_annual','import_export','import_export_annual']
df

import pymysql
     
# 連接到 MySQL
db = pymysql.connect(host="localhost", user="root", 
                     passwd="test", db="trade",  charset='utf8')

df.to_sql('trade_import_export2', 
                con = db, 
                flavor='mysql', 
                if_exists='append')

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


## Apple Crawler

In [6]:
import requests
from bs4 import BeautifulSoup
res = requests.get('http://www.appledaily.com.tw/realtimenews/section/new/')
soup = BeautifulSoup(res.text)

ary = []
for rec in soup.select('.rtddt a'):
    ary.append([rec.select('h1')[0].text, rec.select('h2')[0].text, rec.select('time')[0].text])
    
import pandas
df = pandas.DataFrame(ary)
df

C:\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


,0,1,2
0,南市空污全國第一 環保局：7成污染源自境...(0),政治,14:50
1,憂國際處境 呂秀蓮籲蔡英文：創造2017...(0),政治,14:50
2,中國發射「碳衛星」 觀測二氧化碳循環(0),國際,14:49
3,Fed害慘亞幣 韓元連7貶破1200大關(0),財經,14:48
4,色男潛入檳榔攤 鹹豬手襲臀檳榔西施(0),社會,14:47
5,皮草殘酷真相：水貂肛門過電 活剝貂皮(0),動物,14:47
6,​蔡依林錦榮分手 網友意外扯出這個男人(0),娛樂,14:47
7,我被外交掮客勒索50億(347),論壇,14:44
8,震驚！ 女警執勤遭擄 趴引擎蓋狂飆2K遭...(0),社會,14:44
9,【壹週刊】VJ戀破局！ 錦榮這段話早露...(0),娛樂,14:44


In [7]:
df.to_excel('news.xlsx')